In [4]:
!apt-get -qq install -y sox ffmpeg && pip install --quiet sox
!git clone --quiet https://github.com/harvard-edge/multilingual_kws
import sys
sys.path.append("/content/multilingual_kws/")

In [17]:
from multilingual_kws.embedding import batch_streaming_analysis as sa
import pickle

In [7]:
from google.colab import files
uploaded = files.upload()

Saving mask_stream.mp3 to mask_stream.mp3


In [10]:
!ffmpeg -hide_banner -loglevel error -y -i mask_stream.mp3 -acodec pcm_s16le -ac 1 -ar 16000 mask_stream.wav

In [11]:
!unzip -q masiki_model.zip

In [14]:
!touch empty.txt

In [15]:
target_word = "masiki"
flags = sa.StreamFlags(
    wav="/content/mask_stream.wav",
    ground_truth="/content/empty.txt",
    target_keyword=target_word,
    detection_thresholds=[0.9],
    average_window_duration_ms=100,
    suppression_ms=500,
    time_tolerance_ms=750, #only used when graphing
)
streamtarget = sa.StreamTarget(
    target_lang="luganda",
    target_word=target_word,
    model_path="/content/xfer_epochs_4_bs_64_nbs_2_val_acc_1.00_target_mask/",
    destination_result_pkl="/content/results.pkl",
    destination_result_inferences="/content/inferences.npy",
    stream_flags=[flags],
)

In [16]:
sa.eval_stream_test(streamtarget)

SAVING results TO
 /content/results.pkl
SAVING inferences TO
 /content/inferences.npy
building spectrograms
inferences complete
results for 0.90
No ground truth yet, 35false positives


In [18]:
with open("/content/results.pkl", 'rb') as fh:
  results = pickle.load(fh)

In [25]:
results["masiki"][0][1][0.9][0]

[['masiki', 9060],
 ['masiki', 87680],
 ['masiki', 145460],
 ['masiki', 168460],
 ['masiki', 193620],
 ['masiki', 240960],
 ['masiki', 244680],
 ['masiki', 371240],
 ['masiki', 408680],
 ['masiki', 429280],
 ['masiki', 432020],
 ['masiki', 469280],
 ['masiki', 471240],
 ['masiki', 488600],
 ['masiki', 557360],
 ['masiki', 643200],
 ['masiki', 714680],
 ['masiki', 767480],
 ['masiki', 799340],
 ['masiki', 811720],
 ['masiki', 855500],
 ['masiki', 863180],
 ['masiki', 865300],
 ['masiki', 873320],
 ['masiki', 924360],
 ['masiki', 1030080],
 ['masiki', 1106680],
 ['masiki', 1199280],
 ['masiki', 1200600],
 ['masiki', 1228260],
 ['masiki', 1243620],
 ['masiki', 1310240],
 ['masiki', 1321400],
 ['masiki', 1323880],
 ['masiki', 1386420]]